Importing packages for python, tensorflow for neural network and custom kernals from Kernals.ipynb

In [14]:
import tensorflow as tf
%run Kernels.ipynb

Defining a custom layer to use the kernels in Kernels.ipynb. Kernel is parsed in as an argument. 
Code repurposed from in-class demonstration. 

In [15]:
# Define a custom layer for use with defined kernels
class CustomConvLayer(tf.keras.layers.Layer):
    def __init__(self, kernel):
        super(CustomConvLayer, self).__init__()
        # set the size of the kernel (3x3)
        # 3x3 selected as it takes less processing time, reduces overfitting, has higher spatial resolution
        self.kernel = tf.constant(kernel,dtype=tf.float32)
        # kernel shape - height, width, input channels (1 as greyscale), output channels (1 as greyscale)
        self.kernel = tf.reshape(self.kernel, [3, 3, 1, 1])

    # Apply custom kernel to the convolution layer. 
    def call(self, inputs):
        return tf.nn.conv2d(inputs, self.kernel, strides=1, padding='SAME')

Custom layer for canny edge detection. 
Complex method, first applies smoothing then sobel kernels from Kernels.ipynb in order to find gradients in the X and Y directions. 
![Example of sobel application](IMAGES/Sobel%20chicken.png)
Gradient magnitude measures how strong the change in image intensity is. A number that quantified the strength of a change.
https://pyimagesearch.com/2021/05/12/image-gradients-with-opencv-sobel-and-scharr/
Hysteresis takes two images and combines them to create an intermediate image (in this case, X and Y sobel)
Non-maximal suppression finds the angle of an edge. If a pixel is non-maximal in this line it can be removed. 
https://www.homepages.ucl.ac.uk/~zceeg99/imagepro/three.pdf
Image tensors contain the value of each pixel
https://observablehq.com/@jeyabbalas/image-tensors
![Example of edge detection application](IMAGES/Edge%20Chicken.jpeg)

In [16]:
# Define a custom layer for canny edge detection. 
class CannyEdgeLayer(tf.keras.layers.Layer):

    # low = lower threshold for hysteresis
    # high = upper threshold for hysteresis
    def __init__(self, low=0.1, high=0.3, **kwargs):
        super(CannyEdgeLayer, self).__init__(**kwargs)
        self.low = low
        self.high = high

    def build(self, input_shape):
        # define kernals for gaussian blurring and sobels. 
        self.sobel_x = sobel_kernel_x
        self.sobel_y = sobel_kernel_y
        # [height, width, number of channels (1 as greyscale image), output channels (1 as greyscale)]
        self.sobel_x = tf.reshape(self.sobel_x, [3, 3, 1, 1])
        self.sobel_y = tf.reshape(self.sobel_y, [3, 3, 1, 1])

        gaussian_kernel = gaussian_kernel
        self.gaussian_kernel = tf.reshape(gaussian_kernel, [3, 3, 1, 1])
        super(CannyEdgeLayer, self).build(input_shape)

    def call(self, inputs):
        # apply canny edge detection to the input image
        # blur/smooth image to reduce noise
        blurred = tf.nn.conv2d(inputs, self.gaussian_kernel, strides=1, padding='SAME')

        # find gradients 
        gradient_x = tf.nn.depthwise_conv2d(blurred, self.sobel_x, strides=1, padding='SAME')
        gradient_y = tf.nn.depthwise_conv2d(blurred, self.sobel_y, strides=1, padding ='SAME')

        # calculate edge strength and orientation
        gradient_magnitude = tf.sqrt(tf.square(gradient_x) + tf.square(gradient_y))
        gradient_direction = tf.atan2(gradient_y, gradient_x)

        # apply non-maximum suppression to thin out the edges 
        suppressed = self.non_maximum_suppression(gradient_magnitude, gradient_direction)

        # filter weak and strong edges using thresholds for hysteresis
        strong_edges = tf.where(suppressed >= self.high, 1.0, 0.0)
        weak_edges = tf.where((suppressed >= self.low) & (suppressed < self.high, 1.0, 0.0))

        edges = self.hysteresis_thresholding(strong_edges, weak_edges)

        return edges 
    
    # thin out the edges using non maximum suppression 
    def non_maximum_suppression(self, gradient_magnitude, gradient_direction):
        # convert direction to degrees (currently radians)
        angle = gradient_direction * (180.0 / tf.constant(3.14159))
        # normalise the direction
        angle = tf.where(angle < 0, angle + 180, angle)

        # funtion to change direction of tensors
        # dx = shift along X axis
        # dy = shift along Y axis
        def shifted_tensor(tensor, dx, dy):
            return tf.roll(tensor, shift=[dx, dy], axis = [1, 2])
        
            # dictionary of neighbouring directions to compare against. 
            # Contains angles 0°, 45°, 90° and 135°
        neighbours = {0: (shifted_tensor(gradient_magnitude, 0, -1), shifted_tensor(gradient_magnitude, 0, 1)),
                      45: (shifted_tensor(gradient_magnitude, -1, 1), shifted_tensor(gradient_magnitude, 1, -1)),
                      90: (shifted_tensor(gradient_magnitude, -1, 0), shifted_tensor(gradient_magnitude, 1, 0)),
                      135: (shifted_tensor(gradient_magnitude, -1, -1), shifted_tensor(gradient_magnitude, 1, 1))}
        
        # Keep only the maximum gradient in the edges direction
        suppressed = tf.where(
            # Angle close to 0°
            (tf.logical_and(
                tf.less_equal(angle, 22.5 | tf.greater(angle, 157.5)), # between -22.5° and 22.5° or 157.5° and 180° is a horizontal edge
                (gradient_magnitude >= neighbours[0][0]) & (gradient_magnitude >= neighbours[45][1]) # compare with left and right neighbours
            )) |
            # Angle close to 45°
            (tf.logical_and(
                tf.greater(angle, 22.5) & tf.less_equal(angle, 67.5), # between 22.5° and 67.5° is a diagonal edge \
                (gradient_magnitude >= neighbours[45][0]) & (gradient_magnitude >= neighbours[45][1])
            )) |
            # Angle close to 90°
            (tf.logical_and(
                tf.greater(angle, 67.5) & tf.less_equal(angle, 112.5), # between 67.5° and 112.5° is a vertical edge
                (gradient_magnitude >= neighbours [90][0]) & (gradient_magnitude >= neighbours [90][1])
            ))
            # Angle close to 135°
            (tf.logical_and(
                tf.greater(angle, 112.5) & tf.less_equal(angle, 157.5), # between 112.5 and 157.5 is a diagonal edge /
                (gradient_magnitude >= neighbours[135][0]) & (gradient_magnitude >= neighbours[135][1])
            )),

            # keep pixels if the magnitude is greater than or equal to its neighbours. If not, set the value to 0
            gradient_magnitude, tf.zeros_like(gradient_magnitude)
            )
        
    # Apply hysteresis thresholding to ensure edges stay connected to strong edges. 
    def hysteresis_thresholding(self, strong_edges, weak_edges):
        edges = tf.identity(strong_edges)
        previous_edges = tf.zeros_like(edges)
        # iterate until weak and strong edges converge
        while not tf.reduce_all(tf.equal(edges, previous_edges)):
            previous_edges = edges
            edges = tf.where(weak_edges & self.connected_to_strong(edges), 1.0, edges)
        return edges
    
    # check if weak edges are connected to strong edges in the neighbourhood. Used in hysteresis_thresholding

    def connected_to_strong(self, edges):
        def shifted_tensor(tensor, dx, dy):
            return tf.roll(tensor, shift=[dx, dy], axis=[1, 2])
        
    # Sum of neighbors around each pixel
        neighbor_sum = (
            shifted_tensor(edges, 0, 1) + shifted_tensor(edges, 0, -1) +
            shifted_tensor(edges, 1, 0) + shifted_tensor(edges, -1, 0) +
            shifted_tensor(edges, 1, 1) + shifted_tensor(edges, -1, -1) +
            shifted_tensor(edges, 1, -1) + shifted_tensor(edges, -1, 1)
        )
        return neighbor_sum > 0
    
    # return the configuration for saving and loading
    def get_configuration(self):
        config = super(CannyEdgeLayer, self).get_config()
        config.update({'low_threshold': self.low,
                       'high_threshold': self.high})
        return config